In [1]:
import os
os.chdir('../')
print(os.listdir())
import findspark
findspark.init()
findspark.find()


['.git', '.gitignore', '.idea', 'architecture-design', 'docker', 'drive-download-20211215T213754Z-001.zip', 'drive-download-20211221T190522Z-001.zip', 'evaluated-data', 'models', 'notebooks', 'README.md', 'scripts', 'simulated-data-raw', 'spark-warehouse', 'transformed-data', 'venv']


'D:\\spark\\spark-3.1.2-bin-hadoop2.7\\spark-3.1.2-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [3]:
import pickle
from scripts.generate_data import load_dataset
from scripts.generate_features import transform_data

['.git', '.gitignore', '.idea', 'architecture-design', 'docker', 'drive-download-20211215T213754Z-001.zip', 'drive-download-20211221T190522Z-001.zip', 'evaluated-data', 'models', 'notebooks', 'README.md', 'scripts', 'simulated-data-raw', 'spark-warehouse', 'transformed-data', 'venv']


In [4]:
spark = SparkSession.builder.appName('Spark').master("local[*]").getOrCreate()
# conf = SparkConf().setAppName('Spark').setMaster('local[*]')
# sc = SparkContext.getOrCreate(conf=conf)

### Transform train data

In [5]:
%%time
# generate training features
DIR_INPUT = 'simulated-data-raw/data/'
DIR_OUTPUT = 'transformed-data/training/training_set.parquet'
BEGIN_DATE = '2021-09-01'
END_DATE= '2021-12-01'
df = transform_data(spark, DIR_INPUT, DIR_OUTPUT, BEGIN_DATE, END_DATE)
# df = load_dataset(spark, DIR_INPUT, BEGIN_DATE, END_DATE)

['.idea', '.ipynb_checkpoints', 'docker', 'drive-download-20211215T093959Z-001.zip', 'models', 'notebooks', 'pyspark-analysis.ipynb', 'scripts', 'simulated-data-raw', 'spark-warehouse', 'transformed-data', 'Untitled.ipynb']
Wall time: 5min 25s


### Transform test data

In [6]:
%%time
from datetime import datetime, timedelta, date

# generate test data features
# test period: 2021-12-09 - 2021-12-31
# we will gather data from 2021-11-02 because we want to include historical data into our test set 
# generate training features
DIR_INPUT = 'simulated-data-raw/data/'
DIR_OUTPUT = 'transformed-data/test/test_set.parquet'
# BEGIN_DATE = str((datetime(year=2021, month=12, day=9)-timedelta(days=37)).date())
BEGIN_DATE = '2021-12-09'
END_DATE= '2021-12-31'
df = transform_data(spark, DIR_INPUT, DIR_OUTPUT, BEGIN_DATE, END_DATE)

['.idea', '.ipynb_checkpoints', 'docker', 'drive-download-20211215T093959Z-001.zip', 'models', 'notebooks', 'pyspark-analysis.ipynb', 'scripts', 'simulated-data-raw', 'spark-warehouse', 'transformed-data', 'Untitled.ipynb']
Wall time: 4min 42s


### Feature correlation

In [7]:
import pandas as pd

In [8]:
pd_df = pd.read_parquet('transformed-data/training/training_set.parquet',engine='fastparquet')

In [9]:
pd_df.corr()

,index,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_NIGHT,...,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_TERMINAL_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_TERMINAL_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_TERMINAL_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_MAX_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_MAX_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_MAX_AMOUNT_30DAY_WINDOW
index,1.000000,1.000000,-0.000894,-0.001287,0.000461,0.999994,9.999405e-01,0.015420,0.014347,-7.135270e-03,...,0.004473,0.011704,0.137922,0.500461,0.001183,0.030403,0.218093,0.004091,0.026396,0.085377
TRANSACTION_ID,1.000000,1.000000,-0.000894,-0.001287,0.000461,0.999994,9.999405e-01,0.015420,0.014347,-7.135270e-03,...,0.004473,0.011704,0.137922,0.500461,0.001183,0.030403,0.218093,0.004091,0.026396,0.085377
CUSTOMER_ID,-0.000894,-0.000894,1.000000,-0.000171,-0.000111,-0.000895,-9.010002e-04,-0.001802,-0.001981,-1.018469e-03,...,-0.000273,0.002094,0.003295,0.002823,0.003058,0.005673,0.009888,-0.000174,-0.000772,-0.002086
TERMINAL_ID,-0.001287,-0.001287,-0.000171,1.000000,-0.000882,-0.001284,-1.272772e-03,-0.008144,-0.007214,1.375228e-03,...,0.000237,0.001345,0.001182,0.000823,-0.000732,-0.000469,0.001992,-0.000418,-0.000408,-0.000344
TX_AMOUNT,0.000461,0.000461,-0.000111,-0.000882,1.000000,0.000462,4.653599e-04,0.169677,0.195067,1.124877e-04,...,0.696451,0.002666,0.003583,0.003564,0.001300,0.001717,0.005936,0.759959,0.653880,0.583651
TX_TIME_SECONDS,0.999994,0.999994,-0.000895,-0.001284,0.000462,1.000000,9.999686e-01,0.015416,0.014341,-5.384607e-03,...,0.004473,0.011679,0.137853,0.500196,0.001180,0.030388,0.217973,0.004081,0.026376,0.085331
TX_TIME_DAYS,0.999940,0.999940,-0.000901,-0.001273,0.000465,0.999969,1.000000e+00,0.015426,0.014348,-8.057511e-07,...,0.004475,0.011635,0.137792,0.500147,0.001174,0.030369,0.217943,0.004064,0.026358,0.085319
TX_FRAUD,0.015420,0.015420,-0.001802,-0.008144,0.169677,0.015416,1.542595e-02,1.000000,0.968666,4.053578e-04,...,0.033356,-0.000802,0.004908,0.014506,0.000114,0.000586,0.004972,0.147970,0.140486,0.112403
TX_FRAUD_SCENARIO,0.014347,0.014347,-0.001981,-0.007214,0.195067,0.014341,1.434779e-02,0.968666,1.000000,-5.479927e-05,...,0.033594,-0.000720,0.004580,0.014131,-0.000357,0.000161,0.004689,0.173728,0.169158,0.135636
TX_DURING_NIGHT,-0.007135,-0.007135,-0.001018,0.001375,0.000112,-0.005385,-8.057511e-07,0.000405,-0.000055,1.000000e+00,...,0.000205,-0.002059,-0.004535,-0.004789,-0.000611,-0.001766,-0.002115,-0.001396,-0.001924,-0.001672
